<a href="https://colab.research.google.com/github/partizanos/computational_finance/blob/master/TP12_Comp_Finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Series 12
Introduction to Computational Finance
DIMITRIS PROIOS

## Bonds
Using the same bond as last week, and assuming the required yield is 10%: A bond with
3 years maturity, a face value of 100$ and coupons of 10% paid semi-annually:

|time [years] | coupon [CHF] 
|---|---| 
| 0.5 | 5
| 1 | 5
| 1.5 | 5
| 2 | 5
|2.5 | 5
| 3 | 5


- What is the present value of the bond ?

**Answer**

Value of bonds can be calculated with this formula:
$B = (\sum c e^{rt}) + P e^{rt}$

By replacing with the data of our table the formula becomes:

$B = 5 e^{-0.1 * 0.5} + 5 e^{-0.1* 1} +5 e^{-0.1 * 1.5} +5 e^{-0.1* 2} +5 e^{-0.1 * 2.5}  + 105 e^{-0.1 * 3}$


In [2]:
import math

def present_value(face_value, coupon, periods, rate):
    total_pv = 0
    coupon_values = [coupon / math.pow(- (1 + rate), n) for n in range(1, periods+1)]
    total_pv += sum(coupon_values)
 
    last_payment = face_value / math.pow(- (1 + rate), periods)
    total_pv += last_payment
 
    return total_pv


face_value = 100
coupon = 5
periods = 6
rate = 0.05

present_value(face_value, coupon, periods, rate)

74.0025528261911

- Compute the duration and the convexity of the bond.
- Estimate (with and without convexity) what would be the effect of a x% parallel
shift of the yield curve when x equals 0.1% and 1%.
- Compare your answer to the exact values.


If computing the exact value is so simple, why bothering using an approximation through
duration ?